In [1]:
import functools
import numpy as np
import tensorflow as tf
from keras.datasets import cifar100
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects
from keras.initializers import RandomNormal, GlorotNormal
from keras import regularizers

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

169017344/169001437 [==============================] - 4s 0us/step


In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 100),
 (10000, 32, 32, 3),
 (10000, 100),
 (10000, 32, 32, 3),
 (10000, 100))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def vgg16(act):
    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',
                        input_shape=(32, 32, 3), kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(512, kernel_regularizer=regularizers.l2(0.0005)))
    model.add(Activation(act))
    model.add(BatchNormalization())

    model.add(Dropout(0.5))
    model.add(Dense(100))
    model.add(Activation('softmax'))

    return model

In [9]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = vgg16('elu')

batch_size = 128
epochs = 50

top3_acc = functools.partial(tf.keras.metrics.top_k_categorical_accuracy, k=3)

top3_acc.__name__ = 'top3_acc'

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy', top3_acc, 'top_k_categorical_accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 35s 99ms/step - loss: 7.9751 - accuracy: 0.0158 - top3_acc: 0.0450 - top_k_categorical_accuracy: 0.0727 - val_loss: 6.9341 - val_accuracy: 0.0216 - val_top3_acc: 0.0633 - val_top_k_categorical_accuracy: 0.1010
Epoch 2/50
313/313 [==============================] - 30s 96ms/step - loss: 7.0690 - accuracy: 0.0369 - top3_acc: 0.0929 - top_k_categorical_accuracy: 0.1433 - val_loss: 7.0137 - val_accuracy: 0.0312 - val_top3_acc: 0.0848 - val_top_k_categorical_accuracy: 0.1324
Epoch 3/50
313/313 [==============================] - 29s 92ms/step - loss: 6.7313 - accuracy: 0.0522 - top3_acc: 0.1313 - top_k_categorical_accuracy: 0.1937 - val_loss: 6.8694 - val_accuracy: 0.0424 - val_top3_acc: 0.0972 - val_top_k_categorical_accuracy: 0.1451
Epoch 4/50
313/313 [==============================] - 30s 97ms/step - loss: 6.5212 - accuracy: 0.0613 - top3_acc: 0.1546 - top_k_categorical_accuracy: 0.2285 - val_loss: 6.6295 - val_accuracy: 0.0481 - val_to

In [10]:
print(history.history)
# y_pred = np.argmax(model.predict(x_test), axis=1)
# y_true = np.argmax(y_test,axis=1)

# print(y_pred.shape)
# print(y_true.shape)

# print(np.sum(y_pred == y_true) / y_pred.shape[0])

print(model.evaluate(x_test, y_test))

{'loss': [7.673943519592285, 6.968329906463623, 6.67452335357666, 6.4793195724487305, 6.35328483581543, 6.249958515167236, 6.138543128967285, 6.044961929321289, 5.945988655090332, 5.865896701812744, 5.780890464782715, 5.694162845611572, 5.620155334472656, 5.5457329750061035, 5.479309558868408, 5.418554782867432, 5.343653678894043, 5.2700629234313965, 5.204709529876709, 5.149707794189453, 5.085022926330566, 5.029592990875244, 4.9687042236328125, 4.91947603225708, 4.8571085929870605, 4.803508758544922, 4.756048202514648, 4.694848537445068, 4.64919376373291, 4.603182792663574, 4.564877510070801, 4.517166614532471, 4.467426776885986, 4.4146037101745605, 4.385295867919922, 4.353514194488525, 4.301515579223633, 4.264156818389893, 4.223448753356934, 4.191092014312744, 4.155045986175537, 4.114895820617676, 4.081058979034424, 4.046826362609863, 4.00756311416626, 3.969705104827881, 3.94016170501709, 3.902858257293701, 3.8746719360351562, 3.8434200286865234], 'accuracy': [0.020925000309944153, 0.